In [ ]:
## Import pandas as pd and numpy as np and read csv file

In [8]:
import pandas as pd
import numpy as np

dataBin = pd.read_csv('order_brush_order.csv')

*Parsing Time*

In [10]:
from datetime import datetime

In [11]:
dFormat = '%Y-%m-%d %H:%M:%S'
dataBin['year'] = dataBin['event_time'].apply(lambda x: int(datetime.strptime(x, dFormat).strftime("%Y")))
dataBin['month'] = dataBin['event_time'].apply(lambda x:  int(datetime.strptime(x, dFormat).strftime("%m")))
dataBin['date'] = dataBin['event_time'].apply(lambda x: int( datetime.strptime(x, dFormat).strftime("%d")))
dataBin['hour'] = dataBin['event_time'].apply(lambda x: int( datetime.strptime(x, dFormat).strftime("%H")))
dataBin['minute'] = dataBin['event_time'].apply(lambda x: int( datetime.strptime(x, dFormat).strftime("%M")))

Drop the event time, the year(since all of them are 2019), the month (since all of them are 12) and the orderid(they are insignificant)

In [14]:
dataBin = dataBin.drop(['year', 'month', 'event_time', 'orderid'], axis='columns')

In [15]:
dataBin

,shopid,userid,date,hour,minute
0,93950878,30530270,27,0,23
1,156423439,46057927,27,11,54
2,173699291,67341739,27,13,22
3,63674025,149380322,27,13,1
4,127249066,149493217,27,11,37
...,...,...,...,...,...
222745,110868129,193218997,28,23,17
222746,91639906,3541807,28,19,7
222747,29391773,135908070,28,8,17
222748,61556313,27306783,28,10,14
